<a href="https://colab.research.google.com/github/patrickfleith/datapipes/blob/main/Structured_Output_with_Anthropic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Structured Output with Anthropic
In order to build reliable pipelines in which LLMs consistent return output in the same format, we are using a **Structured Output**
- This means that we define a blueprint for the output
- We pass the 'blueprint' to the LLM
- Then the LLM output will confirm to the blueprint.

This 'blueprint' in the LLM jargon is often called a "schema".

**To generate structured outputs with Anthropic, we'll use the library `instructor`**

In [ ]:
!pip install anthropic instructor --quiet

In [ ]:
import anthropic
from google.colab import userdata
from pydantic import BaseModel
from enum import Enum
import instructor

ANTHROPIC_API_KEY = userdata.get('ANTHROPIC_API_KEY')

We use pydantic to create data models.

In [ ]:
class City(str, Enum):
    aria = "Aria"
    kniga = "Kniga"
    aquabah = "Aquabah"
    torini = "Torini"

class Character(BaseModel):
    name: str
    age: int
    city: City
    job: str
    two_sentences_background_story: str
    inventory: list[str]

In [ ]:
antropic_client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)
client = instructor.from_anthropic(antropic_client)

# note that client.chat.completions.create will also work
player = client.messages.create(
    model="claude-3-5-haiku-latest",
    max_tokens=1024,
    messages=[
        {
            "role": "system",
            "content": "You are a helpful table-top RPG gamemaster assistant."},
        {
            "role": "user",
            "content": "Generate an Alchemist character for a low-fantasy RPG campaign. Be creative"},
    ],
    temperature=0.9,
    response_model=Character,
)

Now we can see the `player` object which has been created.

In [ ]:
player

Character(name='Lyra Emberheart', age=35, city=<City.torini: 'Torini'>, job='Alchemist', two_sentences_background_story="Lyra was once a promising court alchemist who fell from grace after a experimental potion went catastrophically wrong, causing a noble's temporary transformation. Now she works as a freelance alchemist, seeking redemption and pushing the boundaries of her craft in secret.", inventory=['leather-bound alchemy journal', 'brass mortar and pestle', 'collection of rare herb vials', 'experimental transmutation kit', 'emergency healing tincture'])

In [ ]:
# You can access individual fields
player.name, player.age, player.city.name

('Lyra Emberheart', 35, 'torini')

Learn more about instructor here: https://python.useinstructor.com/